In [ ]:
import mysql.connector
import pandas as pd

def fetch_patient_info():
    # Establish connection to MySQL Database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Avengers2@",
        database="mock_patient_project"
    )
    # print("Connected to MySQL Database")

    # Create cursor
    cursor = connection.cursor()

    try:
        # Query random medical record number from patient_data table
        query_medical_record_data = "SELECT * FROM patient_data ORDER BY RAND() LIMIT 1"
        cursor.execute(query_medical_record_data)
        medical_record_data = cursor.fetchone()

        # Get column names from cursor description
        column_names = [description[0] for description in cursor.description]

        # Construct DataFrame with appropriate column labels
        patient_info_df = pd.DataFrame([medical_record_data], columns=column_names)

        # Print the DataFrame
        # print("Patient Info DataFrame:")
        # print(patient_info_df)

    finally:
        # Close cursor and connection
        cursor.close()
        connection.close()
        # print("MySQL connection is closed")

    return patient_info_df

In [ ]:
import numpy as np

def generate_percentage():
    # Generate a random number from a normal distribution centered around 0.15
    # with a standard deviation that controls the spread of the bell curve
    mean = 0.015
    std_dev = 0.005  # Adjust this value to control the spread of the bell curve
    percentage = np.random.normal(mean, std_dev)
    
    # Ensure the generated percentage falls within the desired range [0.1, 0.2]
    percentage = max(0.01, min(0.02, percentage))
    
    return percentage

# Call the function to generate the percentage
percentage_to_fetch = generate_percentage()

print("Generated Percentage:", percentage_to_fetch)


Generated Percentage: 0.018552111099258882


In [ ]:
# Set the percentage of records to fetch
percentage_to_fetch = generate_percentage()  # Change this percentage as needed
num_iterations = 5  # Number of iterations

# Establish connection to MySQL Database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Avengers2@",
    database="mock_patient_project"
)
cursor = connection.cursor()

# Get total number of records in patient_data table
cursor.execute("SELECT COUNT(*) FROM patient_data")
total_records = cursor.fetchone()[0]
print("Total Records in Patient Data Table:", total_records)

# Close cursor and connection
cursor.close()
connection.close()

# Calculate the number of records to fetch based on the percentage
num_records_to_fetch = int(total_records * (percentage_to_fetch / 100))

# Create empty list to store patient info dataframes
patient_info_df_1 = []

# Iterate over the number of iterations
for i in range(num_records_to_fetch):
    # Call the function to fetch patient info
    patient_info_df = fetch_patient_info()
    
    # Append the returned dataframe to the list
    patient_info_df_1.append(patient_info_df.head())

# Concatenate all dataframes in patient_info_dfs into a single dataframe
patient_info_dfs = pd.concat(patient_info_df_1, ignore_index=True)

Total Records in Patient Data Table: 100000


In [ ]:
def upload_to_mysql(dataframe, table_name):
    # Establish connection to MySQL Database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Avengers2@",
        database="mock_patient_project"
    )
    cursor = connection.cursor()

    try:
        # Iterate over each row in the DataFrame and insert it into the MySQL table
        for index, row in dataframe.iterrows():
            # Prepare column names string
            columns_str = ', '.join(dataframe.columns)

            # Prepare placeholders for data values
            placeholders = ', '.join(['%s'] * len(row))

            # Construct SQL query for row insertion
            sql = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"

            # Execute SQL query with row data
            cursor.execute(sql, tuple(row))

        # Commit changes
        connection.commit()

    except Exception as e:
        print("Error:", e)
        connection.rollback()

    finally:
        # Close cursor and connection
        cursor.close()
        connection.close()


In [ ]:
upload_to_mysql(patient_info_dfs, 'encounters')
